## Importing Required Libraries

In [3]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

2025-02-10 23:13:58.274109: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 23:13:58.296367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739209438.321669  333516 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739209438.329123  333516 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 23:13:58.354909: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Loading the data

In [ ]:
# Load the data
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv").squeeze()
X_test = pd.read_csv("data/X_test.csv")
y_test = pd.read_csv("data/y_test.csv").squeeze()

## Building the perceptron

In [4]:
# Single Layer Perceptron Model
def create_perceptron(input_size):
    model = models.Sequential([
        layers.Dense(1, input_shape=(input_size,), activation='sigmoid')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.01),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


In [14]:
# Traing the perceptron model

In [5]:
# Train Single Layer Perceptron
input_size = X_train.shape[1]
perceptron = create_perceptron(input_size)
perceptron.fit(
    X_train, 
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test)
)

# Evaluate Perceptron
perceptron_scores = perceptron.evaluate(X_test, y_test)
print(f"Perceptron Test Accuracy: {perceptron_scores[1]:.4f}")


/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-10 23:14:30.696551: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5418 - loss: 0.7248 - val_accuracy: 0.7763 - val_loss: 0.5582
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7755 - loss: 0.5367 - val_accuracy: 0.7800 - val_loss: 0.4997
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7805 - loss: 0.4939 - val_accuracy: 0.7863 - val_loss: 0.4767
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7921 - loss: 0.4684 - val_accuracy: 0.7917 - val_loss: 0.4638
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7902 - loss: 0.4569 - val_accuracy: 0.7873 - val_loss: 0.4592
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7908 - loss: 0.4561 - val_accuracy: 0.7917 - val_loss: 0.4519
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7963 - loss: 0.4422 - val_accuracy: 0.7903 - val_loss: 0.4487
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8021 - loss: 0.4388 - val_accuracy: 0.

## Building the MLP Model

In [7]:
# MLP Model Class
def create_mlp(input_size, hidden_sizes):
    model = models.Sequential()
    
    # Input layer
    model.add(layers.Dense(hidden_sizes[0], input_shape=(input_size,), activation='relu'))
    
    # Hidden layers
    for hidden_size in hidden_sizes[1:]:
        model.add(layers.Dense(hidden_size, activation='relu'))
    
    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [8]:
# Hyperparameter Search
hidden_layer_configs = [[32], [64, 32], [128, 64, 32]]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64]
num_epochs = 30
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def train_and_evaluate(hidden_sizes, learning_rate, batch_size, X_train, y_train, X_test, y_test):
    model = create_mlp(X_train.shape[1], hidden_sizes)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    history = model.fit(
        X_train, y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=0
    )
    
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

In [10]:

# Grid Search with Cross-Validation
best_config = None
best_accuracy = 0

for hidden_sizes in hidden_layer_configs:# Load the data
    for lr in learning_rates:
        for batch_size in batch_sizes:
            accuracies = []
            
            for train_idx, val_idx in kf.split(X_train):
                X_train_fold = X_train.iloc[train_idx]
                X_val_fold = X_train.iloc[val_idx]
                y_train_fold = y_train.iloc[train_idx]
                y_val_fold = y_train.iloc[val_idx]
                
                acc = train_and_evaluate(
                    hidden_sizes, lr, batch_size,
                    X_train_fold, y_train_fold,
                    X_val_fold, y_val_fold
                )
                accuracies.append(acc)
            
            mean_acc = np.mean(accuracies)
            print(f"Config: {hidden_sizes}, LR: {lr}, Batch: {batch_size}, Acc: {mean_acc:.4f}")
            
            if mean_acc > best_accuracy:
                best_accuracy = mean_acc
                best_config = (hidden_sizes, lr, batch_size)

print(f"\nBest Config: {best_config} with Accuracy: {best_accuracy:.4f}")

/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
 

Config: [32], LR: 0.01, Batch: 16, Acc: 0.8027


/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
 

Config: [32], LR: 0.01, Batch: 32, Acc: 0.8021


/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
 

...

Config: [128, 64, 32], LR: 0.0001, Batch: 16, Acc: 0.7977


/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
 

Config: [128, 64, 32], LR: 0.0001, Batch: 32, Acc: 0.8024


/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
 

Config: [128, 64, 32], LR: 0.0001, Batch: 64, Acc: 0.8010

Best Config: ([64, 32], 0.01, 64) with Accuracy: 0.8039


The best model has:
Hidden Layers: [64, 32]
Learning Rate: 0.01
Batch Size: 64
Accuracy: 0.80329 (80.39%)

In [11]:
# Train Final Model with Best Configuration
best_hidden_sizes, best_lr, best_batch_size = best_config
final_model = create_mlp(X_train.shape[1], best_hidden_sizes)
final_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=best_lr),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

final_model.fit(
    X_train, y_train,
    epochs=num_epochs,
    batch_size=best_batch_size,
    validation_data=(X_test, y_test)
)


Epoch 1/30


/home/srihari/Downloads/krishu/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7464 - loss: 0.5225 - val_accuracy: 0.7897 - val_loss: 0.4550
Epoch 2/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7993 - loss: 0.4400 - val_accuracy: 0.7883 - val_loss: 0.4506
Epoch 3/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7925 - loss: 0.4473 - val_accuracy: 0.7893 - val_loss: 0.4514
Epoch 4/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7988 - loss: 0.4282 - val_accuracy: 0.7893 - val_loss: 0.4476
Epoch 5/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8104 - loss: 0.4225 - val_accuracy: 0.7903 - val_loss: 0.4428
Epoch 6/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8039 - loss: 0.4341 - val_accuracy: 0.7867 - val_loss: 0.4524
Epoch 7/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8063 - loss: 0.4324 - val_accuracy: 0.7953 - val_loss: 0.4477
Epoch 8/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7950 - loss: 0.4442 - val_accuracy: 0.7887 - val_

In [12]:
# Final Evaluation
y_pred = (final_model.predict(X_test) >= 0.5).astype(int)
conf_matrix =  confusion_matrix(y_test, y_pred)
confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step

Confusion Matrix:
[[1542  211]
 [ 427  820]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.88      0.83      1753
           1       0.80      0.66      0.72      1247

    accuracy                           0.79      3000
   macro avg       0.79      0.77      0.77      3000
weighted avg       0.79      0.79      0.78      3000



A two-layer MLP (64, 32 neurons) worked better than simpler or deeper models.
A learning rate (0.01) gave better stability and accuracy.
A batch size of 64 was optimal for efficient training.

In [13]:
# Save the model
final_model.save('best_model.h5')
print("\nModel saved successfully!")


Model saved successfully!
